# Train and Test 1D Convolutional Neural Network for GNSS using RNN

Author: Christopher Liu, 11/14/2021

In [1]:
import os, sys
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
import sift_conv1dnet as sconv
from torch.utils.data import TensorDataset, DataLoader 
import matplotlib.pyplot as plt

## Load Data

In [2]:
# Load dart names
in_fpath = 'sift_ml_input.csv'
if os.path.isfile(in_fpath):
     ml_input = pd.read_csv(in_fpath, dtype = {'unit_sources': str, 'dart': str,\
                                             'lat_d': np.float64, 'long_d': np.float64,\
                                             'extra_forecast': str, 'lat_f': np.float64,\
                                             'long_f': np.float64})
else:
    sys.exit("Error: Unit source file cannot be found.")
dart = ml_input['dart'][ml_input.dart.notnull()].tolist()

# Load fq data
npyd = 'npy'
eta = np.load(os.path.join(npyd,'gnss_eta_all.npy'))

# load unit source TS
dfd = 'unit_src_ts'
eta_us = np.zeros((1440,3,31))
for n, name in enumerate(dart):
    eta_us[:,n,:] = pd.read_csv(os.path.join(dfd,'eta_%s.csv' % name))

# Load inversions (weights and ts)
fq_wts = np.load(os.path.join(npyd,'fq_yong_inv_best.npy'))
fq_ts = np.load(os.path.join(npyd,'fq_wt_eta.npy'))

# Split into train, validation,  and test sets
inddir = 'indices'

train_ind = np.loadtxt(os.path.join(inddir,'fq_dart_train_index.txt')).astype(int)
train_runs= np.loadtxt(os.path.join(inddir,'fq_dart_train_runs.txt')).astype(int)

test_ind = np.loadtxt(os.path.join(inddir,'fq_dart_test_index.txt')).astype(int)
test_runs= np.loadtxt(os.path.join(inddir,'fq_dart_test_runs.txt')).astype(int)

valid_ind = np.loadtxt(os.path.join(inddir,'fq_dart_valid_index.txt')).astype(int)
valid_runs= np.loadtxt(os.path.join(inddir,'fq_dart_valid_runs.txt')).astype(int)

eta_tr = np.swapaxes(eta[train_runs, :, :],1,2)
# target_tr = fq_ts[train_ind,:,:360]
target_tr = fq_wts[train_ind,:]

eta_ts = np.swapaxes(eta[test_runs, :, :],1,2)
# target_ts = fq_ts[test_ind,:,:360]
target_ts = fq_wts[test_ind,:]

eta_v = np.swapaxes(eta[valid_runs, :, :],1,2)
# target_v = fq_ts[valid_ind,:,:360]1
target_v = fq_wts[valid_ind,:]

## Scale Data and Load Dataloader

In [3]:
# Convert to tensors. Will need to redo if i want to keep track of run numbers...
batch = 20
shuf = False
scale = True
scaler = MinMaxScaler(feature_range=(-1, 1))

if scale:
    train_x = torch.Tensor(scaler.fit_transform(eta_tr.reshape(-1, eta_tr.shape[-1])).reshape(eta_tr.shape)).cuda()
    test_x = torch.Tensor(scaler.transform(eta_ts.reshape(-1, eta_ts.shape[-1])).reshape(eta_ts.shape)).cuda()
    valid_x = torch.Tensor(scaler.transform(eta_v.reshape(-1, eta_v.shape[-1])).reshape(eta_v.shape)).cuda()
else:   
    train_x = torch.Tensor(eta_tr).cuda()
    test_x = torch.Tensor(eta_ts).cuda()
    valid_x = torch.Tensor(eta_v).cuda()

train_y = torch.Tensor(target_tr).cuda()
test_y = torch.Tensor(target_ts).cuda()
valid_y = torch.Tensor(target_v).cuda()

us_tn = torch.Tensor(eta_us[:360,:,:]).cuda()

# Using the pytorch dataloader
train_dataset = TensorDataset(train_x,train_y)
test_dataset = TensorDataset(test_x,test_y)
valid_dataset = TensorDataset(valid_x,valid_y)

train_dataloader = DataLoader(train_dataset, batch_size = batch, shuffle = shuf, drop_last= True)
test_dataloader = DataLoader(test_dataset, batch_size = batch, shuffle = shuf, drop_last= True)
valid_dataloader = DataLoader(valid_dataset, batch_size = batch, shuffle = shuf, drop_last= True)

In [4]:
if 0:
    plt.figure(1,figsize=(12,6))
    for r in np.arange(62):
        plt.plot(np.arange(512), eta_ts[100,r,:])

## Train

In [5]:
def valid(dataloader, model, loss_fn):
    size = len(dataloader) # number of batches
    valid_model = model.eval()
    valid_loss = 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = valid_model(X)
            
            valid_loss += loss_fn(pred, y).item()
    valid_loss /= size
    
    return valid_loss

In [6]:
# Set Device
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Set random seed
torch.random.manual_seed(100) #for ae/de, optimizer gets stuck for seed = 100

# Specify model, loss function, and optimizer.

nsources = 31 # Number of unit sources used in inversion
model = sconv.Conv1DNN_GNSS_RNN(62*3, nsources, 5, 128, us_tn).to(device)

loss_func = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Using cuda device


In [ ]:
epochs = 500
nbatches = len(train_dataloader)
train_loss_array = np.zeros(epochs)
test_loss_array = np.zeros(epochs)
valid_loss_array = np.zeros(epochs)

for t in range(epochs):
    train_loss = 0.0
    
    for batch, (X, y) in enumerate(train_dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_func(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # keep track of training loss
        train_loss += loss.item()
    
    # Calculating the batch-averaged loss
    avg_train_loss = train_loss/nbatches
    avg_train_loss_nbn = valid(train_dataloader, model, loss_func)
    avg_valid_loss = valid(valid_dataloader, model, loss_func)
    avg_test_loss = valid(test_dataloader, model, loss_func)
    model.train(True) #Do i need this?
        
    # every 50 epochs, print test error. Adjust print frequency 
    # depending on epoch size
    if (t+1) % 25 == 0:
        print('Epoch: %s' % str(t+1))
        print('------')
        print(f"Avg Train loss: {avg_train_loss:>8f} \n")
        print(f"Avg Train loss w/ eval: {avg_train_loss_nbn:>8f} \n")
        print(f"Avg Validation loss: {avg_valid_loss:>8f} \n")
        print(f"Avg Test loss: {avg_test_loss:>8f} \n")
    
    train_loss_array[t] = avg_train_loss
    valid_loss_array[t] = avg_valid_loss
    test_loss_array[t] = avg_test_loss

Epoch: 25
------
Avg Train loss: 44.556522 

Avg Train loss w/ eval: 44.396184 

Avg Validation loss: 41.147537 

Avg Test loss: 37.516287 

Epoch: 50
------
Avg Train loss: 44.529735 

Avg Train loss w/ eval: 44.394681 

Avg Validation loss: 41.140118 

Avg Test loss: 37.498160 

Epoch: 75
------
Avg Train loss: 30.716856 

Avg Train loss w/ eval: 30.738219 

Avg Validation loss: 31.869185 

Avg Test loss: 29.341451 

Epoch: 100
------
Avg Train loss: 26.454847 

Avg Train loss w/ eval: 27.312397 

Avg Validation loss: 26.050387 

Avg Test loss: 26.302601 

Epoch: 125
------
Avg Train loss: 24.207828 

Avg Train loss w/ eval: 23.761765 

Avg Validation loss: 24.032049 

Avg Test loss: 23.526025 

Epoch: 150
------
Avg Train loss: 23.129676 

Avg Train loss w/ eval: 22.991162 

Avg Validation loss: 24.606351 

Avg Test loss: 23.207116 

Epoch: 175
------
Avg Train loss: 22.840455 

Avg Train loss w/ eval: 22.850827 

Avg Validation loss: 24.692442 

Avg Test loss: 22.985175 

Epoch: 20

## Plot batch-averaged MSE versus epochs

In [ ]:
plt.figure(figsize=(14,10))
plt.plot(train_loss_array, label='Train Loss')
plt.plot(valid_loss_array, label='Valid. Loss')
plt.plot(test_loss_array, label='Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Batch-Avg MSE Error')
plt.legend()
#plt.savefig('gnss_fixed_split_s100.png')

## Output results for plotting

Use model to predict test, validiation, and training data.

In [ ]:
# Return intermediate layer output
# See https://discuss.pytorch.org/t/how-can-l-load-my-best-model-as-a-feature-extractor-evaluator/17254/5
def get_activation(name):
    def hook(model, input, output):
        activations[name] = output.detach()
    return hook

In [ ]:
model.eval() # important to disable dropout layers
with torch.no_grad():
    pred_test = model(test_x)
    pred_train = model(train_x)
    pred_valid = model(valid_x)
    print(loss_func(pred_test,test_y).item())

In [ ]:
activations = {}
model.eval() # important to disable dropout/batchnorm layers
with torch.no_grad():
    h1 = model.relu.register_forward_hook(get_activation('test'))
    pred_test = model(test_x)
    h1.remove()
    h2 = model.relu.register_forward_hook(get_activation('train'))
    pred_train = model(train_x)
    h2.remove()
    h3 = model.relu.register_forward_hook(get_activation('valid'))
    pred_valid = model(valid_x)
    h3.remove()
    print(loss_func(pred_test,test_y).item())

In [ ]:
r = 160
plt.figure(figsize = (24,10))
plt.subplot(3,1,1)
plt.plot(pred_test.detach().numpy()[r,0,:], label = 'Predicted')
plt.plot(target_ts[r,0,:], label = 'True')
plt.legend()
plt.subplot(3,1,2)
plt.plot(pred_test.detach().numpy()[r,1,:])
plt.plot(target_ts[r,1,:])
plt.subplot(3,1,3)
plt.plot(pred_test.detach().numpy()[r,2,:])
plt.plot(target_ts[r,2,:])

#### Output results as .npy

In [ ]:
np.save(os.path.join(npyd,'fq_conv1d_gnss_wts_test_400_rnn_1l.npy'), pred_test.detach().numpy())
np.save(os.path.join(npyd,'fq_conv1d_gnss_wts_train_400_rnn_1l.npy'), pred_train.detach().numpy())
np.save(os.path.join(npyd,'fq_conv1d_gnss_wts_valid_400_rnn_1l.npy'), pred_valid.detach().numpy())

In [ ]:
np.save(os.path.join(npyd,'fq_conv1d_gnss_wts_test_ts_400_rnn_1l.npy'), activations['test'].detach().numpy())
np.save(os.path.join(npyd,'fq_conv1d_gnss_wts_train_ts_400_rnn_1l.npy'), activations['train'].detach().numpy())
np.save(os.path.join(npyd,'fq_conv1d_gnss_wts_valid_ts_400_rnn_1l.npy'), activations['valid'].detach().numpy())

#### Or alternatively output as a .csv file if you wish to use MATLAB to plot results instead

In [ ]:
np.savetxt(os.path.join(npyd,'fq_conv1d_wts_test_300.csv'),pred_test.detach().numpy(), delimiter=',')
np.savetxt(os.path.join(npyd,'fq_conv1d_wts_train_300.csv'),pred_train.detach().numpy(), delimiter=',')
np.savetxt(os.path.join(npyd,'fq_conv1d_wts_valid_300.csv'),pred_valid.detach().numpy(), delimiter=',')

# Output the model
See https://pytorch.org/tutorials/beginner/basics/saveloadrun_tutorial.html for more details such as loading saved models

In [ ]:
# Save the model weights and structure
torch.save(model, 'siftconv1d_model.pth')

# Save ONLY the model weights
torch.save(model.state_dict(), 'siftconv1d_model_wts.pth')

## Load Models

In [ ]:
# Loading the output with weights and structure
model_st = torch.load('siftconv1d_model.pth')
model_st.eval()

# Compare model prediction from above with prediction from the loaded model
# We expect the loss to be 0 if the model was saved and loaded correctly
print(loss_func(model_st(test_x),pred_test).item())

In [ ]:
# Loading the output with ONLY weights
model_wt = sconv.Conv1DNN(3, 31).to(device)
model_wt.load_state_dict(torch.load('siftconv1d_model_wts.pth'))
model_wt.eval()

# Compare model prediction from above with prediction from the loaded model
# We expect the loss to be 0 if the model was saved and loaded correctly
print(loss_func(model_wt(test_x),pred_test).item())